<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b38ead2688e92f30fbf5c9f94e959dcc9d6da44af18c14b555ed1765ed3dccbb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-15 09:43:22
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: 39.86 K (0.29%)
Current PnL: -22.77 L (-15.17%)
CY Booked + Current PnL: -10.87 L (-7.24%)
-------------------
Total profit:  1.35 L
Total loss:  -24.12 L
-------------------
Total Booked + Current PnL: 16.11 L (13.09%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.58%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.96 L (64.85%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.09%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,57.0,X-LC,3.71,213720.0,9220.0,14084.0,0.50,4.51,6.59,11.39,10.0,0.65,1.55,13.86,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,99.15,56.0,M-SC,1.49,85805.0,-14972.0,15067.0,1.49,-14.86,17.56,0.09,245.0,-0.99,0.62,12.73,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-0.10,54.0,X-MC,1.05,209520.0,17778.0,19988.0,0.37,9.27,9.54,19.69,99.0,0.89,1.52,16.70,XY25,NTT,AC
33,ICICIGI,2260.25,-14.77,64.0,X-MC,1.64,146416.0,10283.0,20835.0,6.67,7.55,14.23,22.86,91.0,0.49,1.06,21.12,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-14.04,48.0,X-LC,2.97,214921.0,4555.0,24222.0,0.13,2.17,11.27,13.68,37.0,0.19,1.56,19.05,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,ICICIGI,2260.25,-14.77,64.0,X-MC,1.64,146416.0,10283.0,20835.0,6.67,7.55,14.23,22.86,91.0,0.49,1.06,21.12,X40,ATH,INSURANCE
47,LTIM,7230.20,1.16,67.0,H-LC,3.37,250852.0,4701.0,74503.0,1.93,1.91,29.70,32.18,16.0,0.06,1.82,38.64,X200,ATH,IT
25,GREENPANEL,537.00,215.89,48.0,M-SC,3.87,141056.0,-42022.0,120998.0,1.63,-22.95,85.78,43.14,230.0,-0.35,1.02,32.68,XY24,NTT,MISC
76,TTKPRESTIG,770.00,99.15,56.0,M-SC,1.49,85805.0,-14972.0,15067.0,1.49,-14.86,17.56,0.09,245.0,-0.99,0.62,12.73,OX40N,NTT,DURABLES
43,JIOFIN,387.00,-1.88,52.0,X-LC,12.22,266557.0,-808.0,66266.0,1.42,-0.30,24.86,24.48,48.0,-0.01,1.94,54.50,XY24,BTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.00,-23.29,39.0,X-MC,2.13,131950.0,-3838.0,46592.0,-2.37,-2.83,35.31,31.48,107.0,-0.08,0.96,8.95,X40,ATH,INSURANCE
15,CAMPUS,393.00,-23.71,55.0,M-SC,2.71,158893.0,-8882.0,64717.0,-1.72,-5.29,40.73,33.28,210.0,-0.14,1.15,23.15,XY24,NTT,FOOTWEAR
73,TCS,4389.97,-30.29,43.0,X-LC,12.24,277234.0,-68722.0,135429.0,-1.56,-19.86,48.85,19.28,1.0,-0.51,2.01,2.49,X40,ATH,IT
7,ATULAUTO,844.00,3629.73,50.0,M-SC,4.74,167603.0,-29529.0,119350.0,-1.48,-14.98,71.21,45.57,236.0,-0.25,1.22,18.83,XY24,NTT,AUTO
45,KPIGREEN,731.05,-5.95,42.0,H-SC,6.43,112241.0,-13056.0,71980.0,-1.22,-10.42,64.13,47.03,141.0,-0.18,0.82,40.54,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,68.0,M-SC,17.31,148059.0,-1161.0,145335.0,-0.16,-0.78,98.16,96.62,208.0,-0.01,1.08,60.23,XY25,NTT,FINANCE
59,SAIL,228.00,41.14,46.0,M-MC,12.25,223051.0,-1911.0,169563.0,0.59,-0.85,76.02,74.53,192.0,-0.01,1.62,31.13,XY24,BTT,STEEL
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.27,123554.0,218.0,131412.0,-0.15,0.18,106.36,106.72,119.0,0.00,0.90,23.60,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,105.37,58.0,H-SC,11.50,129951.0,-12528.0,32046.0,1.16,-8.79,24.66,13.70,163.0,-0.39,0.94,53.38,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,99.15,56.0,M-SC,1.49,85805.0,-14972.0,15067.0,1.49,-14.86,17.56,0.09,245.0,-0.99,0.62,12.73,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,1.84,220797.0,-48870.0,85206.0,-0.04,-18.12,38.59,13.47,138.0,-0.57,1.60,12.27,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,43.0,H-SC,2.10,139955.0,-35948.0,77969.0,-0.10,-20.44,55.71,23.89,149.0,-0.46,1.02,20.87,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.89,42.0,H-MC,7.03,101650.0,-30185.0,72405.0,0.28,-22.90,71.23,32.03,98.0,-0.42,0.74,13.86,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,47.0,M-LC,6.32,205232.0,2062.0,40061.0,0.77,1.01,19.52,20.74,55.0,0.05,1.49,6.61,XY25,NTT,FINANCE
31,HINDZINC,730.22,35.55,68.0,M-LC,8.87,221198.0,16122.0,96442.0,0.56,7.86,43.60,54.89,52.0,0.17,1.61,33.85,X5K,ATH,METALS
85,WIPRO,420.00,-12.43,53.0,M-LC,5.69,154020.0,3075.0,106382.0,1.34,2.04,69.07,72.51,53.0,0.03,1.12,8.18,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,53.0,M-LC,9.46,156826.0,15986.0,28511.0,0.68,11.35,18.18,31.59,66.0,0.56,1.14,40.85,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,60.0,H-MC,12.67,192188.0,12380.0,100034.0,0.53,6.89,52.05,62.52,88.0,0.12,1.40,43.32,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,35.55,68.0,M-LC,8.87,221198.0,16122.0,96442.0,0.56,7.86,43.60,54.89,52.0,0.17,1.61,33.85,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,60.0,H-MC,12.67,192188.0,12380.0,100034.0,0.53,6.89,52.05,62.52,88.0,0.12,1.40,43.32,XR,NTT,BANKS
85,WIPRO,420.00,-12.43,53.0,M-LC,5.69,154020.0,3075.0,106382.0,1.34,2.04,69.07,72.51,53.0,0.03,1.12,8.18,XR,NTT,IT
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.27,123554.0,218.0,131412.0,-0.15,0.18,106.36,106.72,119.0,0.00,0.90,23.60,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-49.25,29.0,M-SC,4.05,85845.0,-5654.0,84403.0,-0.81,-6.18,98.32,86.07,223.0,-0.07,0.62,24.66,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,3.31,143657.0,-132866.0,242938.0,0.08,-48.05,169.11,39.81,54.0,-0.55,1.04,0.08,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51421.0,-13585.0,164511.0,0.28,-20.90,319.93,232.18,198.0,-0.08,0.37,0.28,XY24,NTT,MISC
28,HAVELLS,2069.16,-8.45,31.0,X-MC,1.54,226855.0,-20876.0,95937.0,0.35,-8.43,42.29,30.30,92.0,-0.22,1.65,3.74,X40,ATH,ELECTRICAL
12,BATAINDIA,2096.00,-38.15,33.0,X-SC,4.43,92545.0,-36125.0,81421.0,-0.82,-28.08,87.98,35.20,219.0,-0.44,0.67,6.22,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-16.17,33.0,X-MC,5.68,176612.0,-23894.0,121050.0,0.43,-11.92,68.54,48.45,90.0,-0.20,1.28,1.84,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.47,43.0,H-LC,0.90,155075.0,-31020.0,78499.0,-0.19,-16.67,50.62,25.51,15.0,-0.40,1.13,14.28,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.10,54.0,X-MC,1.05,209520.0,17778.0,19988.0,0.37,9.27,9.54,19.69,99.0,0.89,1.52,16.70,XY25,NTT,AC
28,HAVELLS,2069.16,-8.45,31.0,X-MC,1.54,226855.0,-20876.0,95937.0,0.35,-8.43,42.29,30.30,92.0,-0.22,1.65,3.74,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-14.77,64.0,X-MC,1.64,146416.0,10283.0,20835.0,6.67,7.55,14.23,22.86,91.0,0.49,1.06,21.12,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-9.37,54.0,X-LC,2.26,278834.0,13408.0,43387.0,1.37,5.05,15.56,21.40,11.0,0.31,2.03,12.03,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,3.31,143657.0,-132866.0,242938.0,0.08,-48.05,169.11,39.81,54.0,-0.55,1.04,0.08,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51421.0,-13585.0,164511.0,0.28,-20.90,319.93,232.18,198.0,-0.08,0.37,0.28,XY24,NTT,MISC
82,VBL,671.64,-21.70,38.0,X-LC,10.69,282124.0,-33718.0,146394.0,0.14,-10.68,51.89,35.67,5.0,-0.23,2.05,1.70,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-16.17,33.0,X-MC,5.68,176612.0,-23894.0,121050.0,0.43,-11.92,68.54,48.45,90.0,-0.20,1.28,1.84,X40N,ATH,FINANCE
73,TCS,4389.97,-30.29,43.0,X-LC,12.24,277234.0,-68722.0,135429.0,-1.56,-19.86,48.85,19.28,1.0,-0.51,2.01,2.49,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51421.0,-13585.0,164511.0,0.28,-20.90,319.93,232.18,198.0,-0.08,0.37,0.28,XY24,NTT,MISC
55,RELAXO,1176.00,-41.23,35.0,X-SC,3.74,80173.0,-64987.0,140920.0,-0.13,-44.77,175.77,52.31,136.0,-0.46,0.58,7.32,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-38.15,33.0,X-SC,4.43,92545.0,-36125.0,81421.0,-0.82,-28.08,87.98,35.20,219.0,-0.44,0.67,6.22,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.05,47.0,X-SC,1.17,106890.0,4886.0,39036.0,0.99,4.79,36.52,43.06,122.0,0.13,0.78,22.73,X40N,ATH,MISC
32,HONAUT,58357.33,-23.12,44.0,X-SC,2.31,108285.0,-19653.0,66790.0,0.01,-15.36,61.68,36.84,143.0,-0.29,0.79,10.86,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-30.29,43.0,X-LC,12.24,277234.0,-68722.0,135429.0,-1.56,-19.86,48.85,19.28,1.0,-0.51,2.01,2.49,X40,ATH,IT
39,INFY,2275.00,-18.52,51.0,X-LC,7.77,317349.0,4461.0,167211.0,-0.22,1.43,52.69,54.87,3.0,0.03,2.31,8.17,X40,BTT,IT
41,ITC,452.00,-39.61,43.0,X-LC,2.83,195404.0,-4734.0,25617.0,0.71,-2.37,13.11,10.44,4.0,-0.18,1.42,3.07,X40,NTT,FMCG
82,VBL,671.64,-21.70,38.0,X-LC,10.69,282124.0,-33718.0,146394.0,0.14,-10.68,51.89,35.67,5.0,-0.23,2.05,1.70,X40N,ATH,FMCG
1,ABB,7934.00,-40.94,43.0,H-LC,7.43,246792.0,-14827.0,134033.0,0.43,-5.67,54.31,45.57,7.0,-0.11,1.79,5.01,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.70,81043.0,-32506.0,72558.0,-0.09,-28.63,89.53,35.27,268.0,-0.45,0.59,98.36,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,40.0,H-SC,9.86,89730.0,-8250.0,29853.0,-0.17,-8.42,33.27,22.05,152.0,-0.28,0.65,30.43,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,105.37,58.0,H-SC,11.50,129951.0,-12528.0,32046.0,1.16,-8.79,24.66,13.70,163.0,-0.39,0.94,53.38,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,9.93,144973.0,-34098.0,99828.0,-0.30,-19.04,68.86,36.71,135.0,-0.34,1.05,43.84,XR,NTT,BANKS
33,ICICIGI,2260.25,-14.77,64.0,X-MC,1.64,146416.0,10283.0,20835.0,6.67,7.55,14.23,22.86,91.0,0.49,1.06,21.12,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,68.0,M-SC,17.31,148059.0,-1161.0,145335.0,-0.16,-0.78,98.16,96.62,208.0,-0.01,1.08,60.23,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,9.93,144973.0,-34098.0,99828.0,-0.30,-19.04,68.86,36.71,135.0,-0.34,1.05,43.84,XR,NTT,BANKS
31,HINDZINC,730.22,35.55,68.0,M-LC,8.87,221198.0,16122.0,96442.0,0.56,7.86,43.60,54.89,52.0,0.17,1.61,33.85,X5K,ATH,METALS
63,SHALBY,327.00,1166.35,53.0,M-SC,22.50,172210.0,-8933.0,54401.0,0.51,-4.93,31.59,25.10,235.0,-0.16,1.25,38.01,XY24,NTT,HEALTHCARE
50,PGHH,17907.65,-26.71,58.0,X-MC,3.04,212190.0,11370.0,56421.0,-0.70,5.66,26.59,33.76,80.0,0.20,1.54,11.15,X40,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,44.01
2,50,75.84


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.35
LC    33.72
MC    21.89
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.64
X40      14.83
XY25     12.15
XR       10.95
X40N     10.09
AR        8.22
OX40N     7.72
X200      1.82
X5K       1.61
SR        1.11
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.60
X-LC    22.59
X-MC    16.14
M-SC    12.70
M-LC     5.36
H-LC     4.74
X-SC     4.61
H-MC     3.82
M-MC     1.62
L-SC     1.44
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-6.01,40.95
IT,13.04,-17.10,80.44
FINANCE,12.20,-12.59,63.63
MISC,6.91,-20.74,87.64
BANKS,6.43,-12.68,71.95
PAINTS,5.61,-19.52,37.33
ELECTRICAL,5.36,-11.72,50.92
INSURANCE,3.87,-3.77,42.56
AUTO,2.84,-46.97,117.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3353644.0,22
XR,1402742.0,14
AR,1197503.0,9
X40,797263.0,10
X40N,638500.0,8
OX40N,583954.0,10
XY25,510964.0,8
SR,268216.0,2
X5K,96442.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3476591.0,25
M-SC,1290626.0,15
X-LC,1131084.0,13
X-MC,1104493.0,11
X-SC,545538.0,6
H-MC,354603.0,3
H-LC,287035.0,3
M-LC,271396.0,4
L-SC,262352.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1211371.0      6
           XR         830080.0      7
           AR         821680.0      5
M-SC       XY24       699755.0      6
X-MC       XY24       588565.0      4
X-LC       X40        496057.0      5
           XY24       309204.0      2
X-SC       X40N       299606.0      4
H-SC       OX40N      273264.0      4
           SR         268216.0      2
X-MC       X40        219785.0      4
X-LC       X40N       217844.0      3
H-LC       AR         212532.0      2
H-MC       XY24       182164.0      1
X-MC       XY25       175093.0      2
L-SC       XR         172312.0      2
M-MC       XY24       169563.0      1
X-SC       XY24       164511.0      1
M-SC       AR         163291.0      2
           OX40N      148245.0      4
           XY25       145335.0      1
           XR         134000.0      2
X-MC       X40N       121050.0      1
X-LC       XY25       107979.0      3
M-LC       XR         106382.0      1
H-MC       XR         100034.0      1
M-LC       X5K         96442.0      1
L-SC       OX40N       90040.0      1
X-SC       X40         81421.0      1
H-LC       X200        74503.0      1
H-MC       OX40N       72405.0      1
H-SC       MH          71980.0      1
L-MC       XR          59934.0      1
L-LC       XY25        42496.0      1
M-LC       XY25        40061.0      1
           XY24        28511.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
